In [1]:
# file kfkd.py
import os
import numpy as np
from pandas.io.parsers import read_csv
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from lasagne import layers
from lasagne.updates import nesterov_momentum
from nolearn.lasagne import NeuralNet
from time import time
import numpy as np
import pickle
from nolearn.lasagne import BatchIterator
import theano
from sklearn.metrics import mean_squared_error

# Increase Python's recursion limit now that net6 is large enough
import sys
sys.setrecursionlimit(10000)

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GRID K520 (CNMeM is disabled, cuDNN 5110)
/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
FTRAIN = '~/data/training.csv'
FTEST = '~/data/test.csv'

In [3]:
# Write function to load the data
# Normalize the target -- coordinates : current grid 96by96
# Normalize the features -- pixels : current range 255

def load(test = False, cols = None):
    """Loads data from FTEST if *test* is True, otherwise from FTRAIN. 
    Pass a list of *cols* if you're only interested in a subset of the target columns"""
    
    fname = FTEST if test else FTRAIN
    df = read_csv(os.path.expanduser(fname)) #Load pandas dataframe
    
    # The Image column has pixel values separated by space
    # convert these values to numpy arrays:
    df['Image'] = df['Image'].apply(lambda im: np.fromstring(im, sep = ' '))
    
    if cols: # get the useful columns out
        df = df[list(cols) + ['Image']]
        
    print(df.count()) # prints the number of values for each column
    df = df.dropna() 
    # drop all rows that have missing values in them. reduce from 7049 partial to 2140 full examples
    
    # make feature matrix X
    # first convert pixel values from pandas 1D series to numpy 2D array
    # normalize pixel valuaes
    X = np.vstack(df['Image'].values)/255. # scale pixel values to [0,1]
    X = X.astype(np.float32) # make sure numeric val are float32
    # X is numpy.ndarray, shape (2140, 9216) == 2140 examples, 96 grid * 96 grid
    
    
    if not test: # only train data has target columns
        
        y = df[df.columns[:-1]].values #last column is X, we want the y targets here
        # y is numpy.ndarray, shape(2140, 30) -- 7049 examples, 30 targets each
        
        y = (y - 48) / 48 # scale target coordinates to [-1.1] (currently 96 by 96)
        X, y = shuffle(X, y, random_state = 42) #shuffle train data
        y = y.astype(np.float32)
        
    else:
        y = None # test data has no targets available
        
    return X,y

X, y = load()
print("X.shape == {}; X.min == {:.3f}; X.max == {:.3f}".format(
    X.shape, X.min(), X.max()))
print("y.shape == {}; y.min == {:.3f}; y.max == {:.3f}".format(
    y.shape, y.min(), y.max()))

left_eye_center_x            7039
left_eye_center_y            7039
right_eye_center_x           7036
right_eye_center_y           7036
left_eye_inner_corner_x      2271
left_eye_inner_corner_y      2271
left_eye_outer_corner_x      2267
left_eye_outer_corner_y      2267
right_eye_inner_corner_x     2268
right_eye_inner_corner_y     2268
right_eye_outer_corner_x     2268
right_eye_outer_corner_y     2268
left_eyebrow_inner_end_x     2270
left_eyebrow_inner_end_y     2270
left_eyebrow_outer_end_x     2225
left_eyebrow_outer_end_y     2225
right_eyebrow_inner_end_x    2270
right_eyebrow_inner_end_y    2270
right_eyebrow_outer_end_x    2236
right_eyebrow_outer_end_y    2236
nose_tip_x                   7049
nose_tip_y                   7049
mouth_left_corner_x          2269
mouth_left_corner_y          2269
mouth_right_corner_x         2270
mouth_right_corner_y         2270
mouth_center_top_lip_x       2275
mouth_center_top_lip_y       2275
mouth_center_bottom_lip_x    7016
mouth_center_b

In [4]:
# When using convolutional layers in Lasagne, we have to prepare the input data such 
# that each sample is no longer a flat vector of 9,216 pixel intensities, 
# but a three-dimensional matrix with shape (c, 0, 1), where c is the number of channels (colors), 
# and 0 and 1 correspond to the x and y dimensions of the input image. In our case, 
# the concrete shape will be (1, 96, 96), because we're dealing with a single (gray) color 
# channel only.

def load2d(test=False, cols=None):
    X, y = load(test=test)
    X = X.reshape(-1, 1, 96, 96)
    return X, y

In [5]:
flip_indices = [
    (0, 2), (1, 3),
    (4, 8), (5, 9), (6, 10), (7, 11),
    (12, 16), (13, 17), (14, 18), (15, 19),
    (22, 24), (23, 25),
    ]

# Let's see if we got it right:
df = read_csv(os.path.expanduser(FTRAIN))
for i, j in flip_indices:
    print("# {} -> {}".format(df.columns[i], df.columns[j]))

# left_eye_center_x -> right_eye_center_x
# left_eye_center_y -> right_eye_center_y
# left_eye_inner_corner_x -> right_eye_inner_corner_x
# left_eye_inner_corner_y -> right_eye_inner_corner_y
# left_eye_outer_corner_x -> right_eye_outer_corner_x
# left_eye_outer_corner_y -> right_eye_outer_corner_y
# left_eyebrow_inner_end_x -> right_eyebrow_inner_end_x
# left_eyebrow_inner_end_y -> right_eyebrow_inner_end_y
# left_eyebrow_outer_end_x -> right_eyebrow_outer_end_x
# left_eyebrow_outer_end_y -> right_eyebrow_outer_end_y
# mouth_left_corner_x -> mouth_right_corner_x
# mouth_left_corner_y -> mouth_right_corner_y


In [6]:
class FlipBatchIterator(BatchIterator):
    flip_indices = [
        (0, 2), (1, 3),
        (4, 8), (5, 9), (6, 10), (7, 11),
        (12, 16), (13, 17), (14, 18), (15, 19),
        (22, 24), (23, 25),
        ]

    def transform(self, Xb, yb):
        Xb, yb = super(FlipBatchIterator, self).transform(Xb, yb)

        # Flip half of the images in this batch at random:
        bs = Xb.shape[0] #number of examples
        indices = np.random.choice(bs, bs // 2, replace=False) #choose half of them randomly
        Xb[indices] = Xb[indices, :, :, ::-1] #flip these guys randomly

        if yb is not None:
            # Horizontal flip of all x coordinates:
            yb[indices, ::2] = yb[indices, ::2] * -1

            # Swap places, e.g. left_eye_center_x -> right_eye_center_x
            for a, b in self.flip_indices:
                yb[indices, a], yb[indices, b] = (
                    yb[indices, b], yb[indices, a])

        return Xb, yb

In [7]:
def float32(k):
    return np.cast['float32'](k)

In [8]:
class AdjustVariable(object):
    def __init__(self, name, start=0.03, stop=0.001):
        self.name = name
        self.start, self.stop = start, stop
        self.ls = None

    def __call__(self, nn, train_history):
        if self.ls is None:
            self.ls = np.linspace(self.start, self.stop, nn.max_epochs)

        epoch = train_history[-1]['epoch']
        new_value = float32(self.ls[epoch - 1])
        getattr(nn, self.name).set_value(new_value)

In [9]:
class EarlyStopping(object):
    def __init__(self, patience=100):
        self.patience = patience
        self.best_valid = np.inf
        self.best_valid_epoch = 0
        self.best_weights = None

    def __call__(self, nn, train_history):
        current_valid = train_history[-1]['valid_loss']
        current_epoch = train_history[-1]['epoch']
        if current_valid < self.best_valid:
            self.best_valid = current_valid
            self.best_valid_epoch = current_epoch
            self.best_weights = nn.get_all_params_values()
        elif self.best_valid_epoch + self.patience < current_epoch:
            print("Early stopping.")
            print("Best valid loss was {:.6f} at epoch {}.".format(
                self.best_valid, self.best_valid_epoch))
            nn.load_params_from(self.best_weights)
            raise StopIteration()

In [10]:
net6 = NeuralNet(
    
    # Specify Architecture
    layers=[
        ('input', layers.InputLayer),
        ('conv1', layers.Conv2DLayer),
        ('pool1', layers.MaxPool2DLayer),
        ('dropout1', layers.DropoutLayer),  # !DROPOUT
        ('conv2', layers.Conv2DLayer),
        ('pool2', layers.MaxPool2DLayer),
        ('dropout2', layers.DropoutLayer),  # !DROPOUT
        ('conv3', layers.Conv2DLayer),
        ('pool3', layers.MaxPool2DLayer),
        ('dropout3', layers.DropoutLayer),  # !DROPOUT
        ('hidden4', layers.DenseLayer),
        ('dropout4', layers.DropoutLayer),  # !DROPOUT
        ('hidden5', layers.DenseLayer),
        ('output', layers.DenseLayer),
        ],
    input_shape=(None, 1, 96, 96),
    conv1_num_filters=32, conv1_filter_size=(3, 3), pool1_pool_size=(2, 2),
    dropout1_p=0.1,  # !DROPOUT PROBABILIY
    conv2_num_filters=64, conv2_filter_size=(2, 2), pool2_pool_size=(2, 2),
    dropout2_p=0.2,  # !
    conv3_num_filters=128, conv3_filter_size=(2, 2), pool3_pool_size=(2, 2),
    dropout3_p=0.3,  # !
    hidden4_num_units=500,
    dropout4_p=0.5,  # !
    hidden5_num_units=500,
    output_num_units=30, output_nonlinearity=None,

    update_learning_rate=theano.shared(float32(0.03)), #Set startup update rate
    update_momentum=theano.shared(float32(0.9)), #Set startup momentum
    
    on_epoch_finished=[
        AdjustVariable('update_learning_rate', start=0.03, stop=0.0001), #Decrease update rate with epoch
        AdjustVariable('update_momentum', start=0.9, stop=0.999), #Increaes momentum with epoch
        EarlyStopping(patience=200) # Early Stopping
        ],
    
    # Regression NOT classification problem
    regression=True,
    
    # Flip half of the images in each batch
    batch_iterator_train=FlipBatchIterator(batch_size=128),
    
    
    max_epochs=3000,
    verbose=1,
    )

X, y = load2d()

left_eye_center_x            7039
left_eye_center_y            7039
right_eye_center_x           7036
right_eye_center_y           7036
left_eye_inner_corner_x      2271
left_eye_inner_corner_y      2271
left_eye_outer_corner_x      2267
left_eye_outer_corner_y      2267
right_eye_inner_corner_x     2268
right_eye_inner_corner_y     2268
right_eye_outer_corner_x     2268
right_eye_outer_corner_y     2268
left_eyebrow_inner_end_x     2270
left_eyebrow_inner_end_y     2270
left_eyebrow_outer_end_x     2225
left_eyebrow_outer_end_y     2225
right_eyebrow_inner_end_x    2270
right_eyebrow_inner_end_y    2270
right_eyebrow_outer_end_x    2236
right_eyebrow_outer_end_y    2236
nose_tip_x                   7049
nose_tip_y                   7049
mouth_left_corner_x          2269
mouth_left_corner_y          2269
mouth_right_corner_x         2270
mouth_right_corner_y         2270
mouth_center_top_lip_x       2275
mouth_center_top_lip_y       2275
mouth_center_bottom_lip_x    7016
mouth_center_b

In [11]:
time0 = time()
net6.fit(X, y)
print(time()-time0)

with open('net6.pickle', 'wb') as f:
    pickle.dump(net6, f, -1)

# Neural Network with 8051502 learnable parameters

## Layer information

  #  name      size
---  --------  ---------
  0  input     1x96x96
  1  conv1     32x94x94
  2  pool1     32x47x47
  3  dropout1  32x47x47
  4  conv2     64x46x46
  5  pool2     64x23x23
  6  dropout2  64x23x23
  7  conv3     128x22x22
  8  pool3     128x11x11
  9  dropout3  128x11x11
 10  hidden4   500
 11  dropout4  500
 12  hidden5   500
 13  output    30

  epoch    trn loss    val loss    trn/val  dur
-------  ----------  ----------  ---------  -----
      1     0.07683     0.03563    2.15602  1.94s
      2     0.01801     0.02381    0.75622  1.91s
      3     0.01108     0.01448    0.76537  1.92s
      4     0.00876     0.01112    0.78752  1.91s
      5     0.00787     0.00869    0.90465  1.91s
      6     0.00739     0.00785    0.94161  1.91s
      7     0.00694     0.00722    0.96177  1.91s
      8     0.00688     0.00723    0.95112  1.91s
      9     0.00663     0.00694    0.95496  1.91s
     10     0.0

    128     0.00442     0.00417    1.05779  1.90s
    129     0.00441     0.00417    1.05634  1.90s
    130     0.00442     0.00418    1.05729  1.90s
    131     0.00441     0.00418    1.05546  1.91s
    132     0.00440     0.00418    1.05366  1.90s
    133     0.00441     0.00418    1.05613  1.90s
    134     0.00440     0.00417    1.05399  1.90s
    135     0.00440     0.00417    1.05445  1.90s
    136     0.00440     0.00417    1.05406  1.90s
    137     0.00440     0.00417    1.05314  1.90s
    138     0.00439     0.00417    1.05311  1.90s
    139     0.00440     0.00417    1.05616  1.90s
    140     0.00439     0.00416    1.05578  1.90s
    141     0.00440     0.00416    1.05639  1.90s
    142     0.00439     0.00417    1.05330  1.90s
    143     0.00439     0.00417    1.05283  1.90s
    144     0.00439     0.00416    1.05506  1.90s
    145     0.00438     0.00416    1.05412  1.90s
    146     0.00438     0.00416    1.05326  1.90s
    147     0.00438     0.00416    1.05286  1.90s


    264     0.00408     0.00387    1.05222  1.91s
    265     0.00409     0.00387    1.05574  1.91s
    266     0.00407     0.00387    1.05339  1.91s
    267     0.00408     0.00386    1.05747  1.91s
    268     0.00406     0.00386    1.05410  1.91s
    269     0.00407     0.00385    1.05622  1.91s
    270     0.00408     0.00385    1.05942  1.91s
    271     0.00408     0.00385    1.06225  1.91s
    272     0.00405     0.00384    1.05524  1.91s
    273     0.00405     0.00384    1.05515  1.91s
    274     0.00405     0.00383    1.05734  1.91s
    275     0.00406     0.00383    1.05957  1.91s
    276     0.00402     0.00382    1.05291  1.91s
    277     0.00404     0.00382    1.05869  1.90s
    278     0.00404     0.00381    1.05862  1.90s
    279     0.00401     0.00381    1.05416  1.90s
    280     0.00401     0.00380    1.05537  1.91s
    281     0.00401     0.00379    1.05864  1.91s
    282     0.00402     0.00379    1.06073  1.91s
    283     0.00401     0.00379    1.05837  1.91s


    394     0.00339     0.00304    1.11501  1.92s
    395     0.00337     0.00303    1.11275  1.92s
    396     0.00339     0.00302    1.12251  1.91s
    397     0.00337     0.00303    1.11456  1.91s
    398     0.00336     0.00302    1.11456  1.91s
    399     0.00336     0.00301    1.11668  1.91s
    400     0.00336     0.00299    1.12152  1.91s
    401     0.00332     0.00299    1.11066  1.91s
    402     0.00336     0.00299    1.12211  1.91s
    403     0.00332     0.00298    1.11143  1.91s
    404     0.00334     0.00297    1.12517  1.91s
    405     0.00333     0.00297    1.12228  1.91s
    406     0.00330     0.00295    1.11750  1.91s
    407     0.00331     0.00295    1.12482  1.91s
    408     0.00327     0.00294    1.11223  1.91s
    409     0.00328     0.00293    1.12277  1.91s
    410     0.00327     0.00292    1.11869  1.91s
    411     0.00329     0.00292    1.12493  1.91s
    412     0.00328     0.00291    1.12998  1.91s
    413     0.00325     0.00289    1.12446  1.91s


    530     0.00269     0.00230    1.17010  1.91s
    531     0.00270     0.00230    1.17576  1.91s
    532     0.00270     0.00231    1.17325  1.91s
    533     0.00272     0.00230    1.18298  1.91s
    534     0.00266     0.00229    1.16486  1.91s
    535     0.00266     0.00229    1.16249  1.91s
    536     0.00265     0.00229    1.15531  1.91s
    537     0.00268     0.00228    1.17510  1.91s
    538     0.00268     0.00228    1.17496  1.91s
    539     0.00269     0.00227    1.18452  1.91s
    540     0.00265     0.00228    1.16125  1.91s
    541     0.00266     0.00227    1.16976  1.91s
    542     0.00266     0.00227    1.16947  1.91s
    543     0.00263     0.00227    1.15918  1.91s
    544     0.00265     0.00226    1.17009  1.91s
    545     0.00265     0.00227    1.16620  1.91s
    546     0.00265     0.00226    1.17136  1.91s
    547     0.00266     0.00226    1.17537  1.91s
    548     0.00264     0.00225    1.17180  1.91s
    549     0.00264     0.00225    1.17350  1.91s


...

   2222     0.00144     0.00120    1.19352  1.91s
   2223     0.00146     0.00120    1.21339  1.91s
   2224     0.00144     0.00120    1.20294  1.92s
   2225     0.00143     0.00120    1.19054  1.92s
   2226     0.00147     0.00120    1.22088  1.92s
   2227     0.00143     0.00120    1.19261  1.92s
   2228     0.00143     0.00120    1.18931  1.91s
   2229     0.00143     0.00120    1.18405  1.91s
   2230     0.00143     0.00120    1.18809  1.91s
   2231     0.00143     0.00120    1.19279  1.91s
   2232     0.00143     0.00120    1.19421  1.91s
   2233     0.00143     0.00120    1.19332  1.91s
   2234     0.00142     0.00120    1.18012  1.91s
   2235     0.00143     0.00120    1.18963  1.91s
   2236     0.00145     0.00120    1.20639  1.91s
   2237     0.00143     0.00120    1.18936  1.91s
   2238     0.00142     0.00120    1.19139  1.91s
   2239     0.00144     0.00119    1.20493  1.91s
   2240     0.00144     0.00120    1.20167  1.91s
   2241     0.00144     0.00120    1.19593  1.91s


   2379     0.00141     0.00117    1.20295  1.91s
   2380     0.00141     0.00117    1.20356  1.91s
   2381     0.00142     0.00117    1.21035  1.91s
   2382     0.00142     0.00117    1.20891  1.91s
   2383     0.00141     0.00117    1.20649  1.91s
   2384     0.00138     0.00117    1.17713  1.91s
   2385     0.00143     0.00118    1.21349  1.91s
   2386     0.00138     0.00118    1.17723  1.91s
   2387     0.00141     0.00117    1.19824  1.91s
   2388     0.00138     0.00117    1.18157  1.91s
   2389     0.00139     0.00117    1.18954  1.91s
   2390     0.00138     0.00117    1.17846  1.91s
   2391     0.00141     0.00118    1.19625  1.91s
   2392     0.00138     0.00118    1.17717  1.91s
   2393     0.00141     0.00117    1.20693  1.91s
   2394     0.00137     0.00117    1.17095  1.91s
   2395     0.00139     0.00117    1.18406  1.91s
   2396     0.00138     0.00117    1.17744  1.91s
   2397     0.00140     0.00117    1.19817  1.91s
   2398     0.00140     0.00117    1.19924  1.91s


   2536     0.00137     0.00115    1.19331  1.92s
   2537     0.00138     0.00115    1.19739  1.92s
   2538     0.00138     0.00115    1.20277  1.91s
   2539     0.00137     0.00115    1.19176  1.91s
   2540     0.00136     0.00115    1.18089  1.91s
   2541     0.00137     0.00115    1.19237  1.91s
   2542     0.00135     0.00115    1.17772  1.91s
   2543     0.00136     0.00115    1.18948  1.91s
   2544     0.00137     0.00115    1.19209  1.91s
   2545     0.00137     0.00115    1.19947  1.91s
   2546     0.00138     0.00115    1.20306  1.91s
   2547     0.00136     0.00115    1.18642  1.91s
   2548     0.00137     0.00115    1.19593  1.91s
   2549     0.00135     0.00115    1.18022  1.91s
   2550     0.00138     0.00115    1.19867  1.91s
   2551     0.00135     0.00115    1.17829  1.91s
   2552     0.00135     0.00115    1.17531  1.91s
   2553     0.00136     0.00115    1.18641  1.91s
   2554     0.00136     0.00114    1.19207  1.91s
   2555     0.00137     0.00114    1.19567  1.91s


   2695     0.00134     0.00113    1.18882  1.91s
   2696     0.00133     0.00113    1.18353  1.91s
   2697     0.00133     0.00113    1.17744  1.92s
   2698     0.00134     0.00113    1.19189  1.91s
   2699     0.00133     0.00113    1.18431  1.91s
   2700     0.00133     0.00112    1.18432  1.91s
   2701     0.00137     0.00112    1.21570  1.91s
   2702     0.00134     0.00112    1.19430  1.91s
   2703     0.00132     0.00112    1.17675  1.92s
   2704     0.00132     0.00112    1.17285  1.92s
   2705     0.00133     0.00112    1.18285  1.92s
   2706     0.00135     0.00112    1.19947  1.91s
   2707     0.00134     0.00112    1.19085  1.91s
   2708     0.00137     0.00113    1.21597  1.91s
   2709     0.00133     0.00113    1.18107  1.91s
   2710     0.00130     0.00113    1.15715  1.91s
   2711     0.00133     0.00113    1.18472  1.91s
   2712     0.00135     0.00113    1.20098  1.91s
   2713     0.00133     0.00113    1.18527  1.91s
   2714     0.00135     0.00113    1.19828  1.91s


   2853     0.00132     0.00111    1.19318  1.92s
   2854     0.00130     0.00111    1.17500  1.91s
   2855     0.00131     0.00111    1.18286  1.91s
   2856     0.00132     0.00111    1.18795  1.91s
   2857     0.00131     0.00111    1.18419  1.91s
   2858     0.00131     0.00111    1.18565  1.91s
   2859     0.00131     0.00111    1.18600  1.91s
   2860     0.00132     0.00111    1.19073  1.91s
   2861     0.00132     0.00111    1.19282  1.91s
   2862     0.00131     0.00111    1.18645  1.91s
   2863     0.00128     0.00110    1.16074  1.91s
   2864     0.00131     0.00110    1.19037  1.91s
   2865     0.00130     0.00110    1.17814  1.91s
   2866     0.00131     0.00110    1.19012  1.91s
   2867     0.00129     0.00110    1.17212  1.91s
   2868     0.00131     0.00110    1.18625  1.91s
   2869     0.00131     0.00110    1.18703  1.91s
   2870     0.00131     0.00110    1.18949  1.91s
   2871     0.00133     0.00110    1.20790  1.91s
   2872     0.00131     0.00110    1.18523  1.92s


In [10]:
net7 = NeuralNet(
    
    # Specify Architecture
    layers=[
        ('input', layers.InputLayer),
        ('conv1', layers.Conv2DLayer),
        ('pool1', layers.MaxPool2DLayer),
        ('dropout1', layers.DropoutLayer),  # !DROPOUT
        ('conv2', layers.Conv2DLayer),
        ('pool2', layers.MaxPool2DLayer),
        ('dropout2', layers.DropoutLayer),  # !DROPOUT
        ('conv3', layers.Conv2DLayer),
        ('pool3', layers.MaxPool2DLayer),
        ('dropout3', layers.DropoutLayer),  # !DROPOUT
        ('hidden4', layers.DenseLayer),
        ('dropout4', layers.DropoutLayer),  # !DROPOUT
        ('hidden5', layers.DenseLayer),
        ('output', layers.DenseLayer),
        ],
    input_shape=(None, 1, 96, 96),
    conv1_num_filters=32, conv1_filter_size=(3, 3), pool1_pool_size=(2, 2),
    dropout1_p=0.1,  # !DROPOUT PROBABILIY
    conv2_num_filters=64, conv2_filter_size=(2, 2), pool2_pool_size=(2, 2),
    dropout2_p=0.2,  # !
    conv3_num_filters=128, conv3_filter_size=(2, 2), pool3_pool_size=(2, 2),
    dropout3_p=0.3,  # !
    hidden4_num_units=1000,
    dropout4_p=0.5,  # !
    hidden5_num_units=1000,
    output_num_units=30, output_nonlinearity=None,

    update_learning_rate=theano.shared(float32(0.03)), #Set startup update rate
    update_momentum=theano.shared(float32(0.9)), #Set startup momentum
    
    on_epoch_finished=[
        AdjustVariable('update_learning_rate', start=0.03, stop=0.0001), #Decrease update rate with epoch
        AdjustVariable('update_momentum', start=0.9, stop=0.999), #Increaes momentum with epoch
        EarlyStopping(patience=200) # Early Stopping
        ],
    
    # Regression NOT classification problem
    regression=True,
    
    # Flip half of the images in each batch
    batch_iterator_train=FlipBatchIterator(batch_size=128),
    
    
    max_epochs=3000,
    verbose=1,
    )

X, y = load2d()

left_eye_center_x            7039
left_eye_center_y            7039
right_eye_center_x           7036
right_eye_center_y           7036
left_eye_inner_corner_x      2271
left_eye_inner_corner_y      2271
left_eye_outer_corner_x      2267
left_eye_outer_corner_y      2267
right_eye_inner_corner_x     2268
right_eye_inner_corner_y     2268
right_eye_outer_corner_x     2268
right_eye_outer_corner_y     2268
left_eyebrow_inner_end_x     2270
left_eyebrow_inner_end_y     2270
left_eyebrow_outer_end_x     2225
left_eyebrow_outer_end_y     2225
right_eyebrow_inner_end_x    2270
right_eyebrow_inner_end_y    2270
right_eyebrow_outer_end_x    2236
right_eyebrow_outer_end_y    2236
nose_tip_x                   7049
nose_tip_y                   7049
mouth_left_corner_x          2269
mouth_left_corner_y          2269
mouth_right_corner_x         2270
mouth_right_corner_y         2270
mouth_center_top_lip_x       2275
mouth_center_top_lip_y       2275
mouth_center_bottom_lip_x    7016
mouth_center_b

In [11]:
time0 = time()
net7.fit(X, y)
print(time()-time0)

with open('net7.pickle', 'wb') as f:
    pickle.dump(net7, f, -1)

# Neural Network with 16561502 learnable parameters

## Layer information

  #  name      size
---  --------  ---------
  0  input     1x96x96
  1  conv1     32x94x94
  2  pool1     32x47x47
  3  dropout1  32x47x47
  4  conv2     64x46x46
  5  pool2     64x23x23
  6  dropout2  64x23x23
  7  conv3     128x22x22
  8  pool3     128x11x11
  9  dropout3  128x11x11
 10  hidden4   1000
 11  dropout4  1000
 12  hidden5   1000
 13  output    30

  epoch    trn loss    val loss    trn/val  dur
-------  ----------  ----------  ---------  -----
      1     0.06542     0.03921    1.66818  2.07s
      2     0.01516     0.02491    0.60847  2.07s
      3     0.00985     0.01667    0.59098  2.07s
      4     0.00788     0.01322    0.59561  2.06s
      5     0.00720     0.01026    0.70118  2.06s
      6     0.00692     0.00981    0.70518  2.08s
      7     0.00669     0.00880    0.76040  2.07s
      8     0.00639     0.00860    0.74328  2.08s
      9     0.00631     0.00832    0.75819  2.08s
     10    

    127     0.00430     0.00418    1.02827  2.07s
    128     0.00426     0.00417    1.02168  2.07s
    129     0.00427     0.00414    1.03077  2.06s
    130     0.00429     0.00418    1.02543  2.06s
    131     0.00428     0.00415    1.03060  2.07s
    132     0.00428     0.00414    1.03240  2.07s
    133     0.00428     0.00413    1.03543  2.08s
    134     0.00427     0.00412    1.03700  2.08s
    135     0.00426     0.00412    1.03330  2.06s
    136     0.00428     0.00412    1.04056  2.06s
    137     0.00427     0.00410    1.04224  2.06s
    138     0.00425     0.00407    1.04294  2.05s
    139     0.00426     0.00408    1.04349  2.06s
    140     0.00426     0.00409    1.04126  2.07s
    141     0.00425     0.00407    1.04317  2.05s
    142     0.00424     0.00407    1.04127  2.06s
    143     0.00424     0.00407    1.04098  2.08s
    144     0.00424     0.00407    1.04340  2.08s
    145     0.00425     0.00405    1.04895  2.07s
    146     0.00421     0.00405    1.03901  2.07s


    265     0.00371     0.00350    1.06151  2.04s
    266     0.00371     0.00350    1.05909  2.04s
    267     0.00370     0.00350    1.05616  2.04s
    268     0.00368     0.00349    1.05625  2.04s
    269     0.00370     0.00347    1.06451  2.04s
    270     0.00370     0.00346    1.06846  2.04s
    271     0.00368     0.00347    1.06190  2.04s
    272     0.00367     0.00346    1.06189  2.04s
    273     0.00367     0.00344    1.06544  2.04s
    274     0.00366     0.00346    1.05913  2.04s
    275     0.00365     0.00346    1.05652  2.04s
    276     0.00365     0.00345    1.05816  2.04s
    277     0.00365     0.00344    1.06061  2.04s
    278     0.00362     0.00342    1.05763  2.04s
    279     0.00361     0.00340    1.06260  2.04s
    280     0.00363     0.00340    1.06570  2.04s
    281     0.00364     0.00341    1.06602  2.04s
    282     0.00364     0.00341    1.06667  2.04s
    283     0.00360     0.00341    1.05823  2.04s
    284     0.00360     0.00338    1.06572  2.04s


    402     0.00286     0.00262    1.09187  2.04s
    403     0.00287     0.00262    1.09666  2.04s
    404     0.00284     0.00261    1.08536  2.04s
    405     0.00284     0.00261    1.08685  2.04s
    406     0.00284     0.00261    1.08723  2.04s
    407     0.00284     0.00258    1.09930  2.04s
    408     0.00284     0.00260    1.09164  2.04s
    409     0.00282     0.00260    1.08287  2.04s
    410     0.00281     0.00257    1.09295  2.04s
    411     0.00281     0.00259    1.08716  2.04s
    412     0.00280     0.00258    1.08468  2.04s
    413     0.00280     0.00257    1.09118  2.04s
    414     0.00280     0.00256    1.09471  2.04s
    415     0.00277     0.00257    1.08038  2.04s
    416     0.00280     0.00255    1.09626  2.04s
    417     0.00281     0.00256    1.09757  2.04s
    418     0.00279     0.00255    1.09308  2.04s
    419     0.00279     0.00256    1.09126  2.04s
    420     0.00277     0.00255    1.08900  2.04s
    421     0.00277     0.00254    1.09222  2.04s


    547     0.00235     0.00216    1.08954  2.04s
    548     0.00240     0.00216    1.11063  2.04s
    549     0.00239     0.00216    1.10645  2.04s
    550     0.00237     0.00215    1.10012  2.04s
    551     0.00237     0.00215    1.10245  2.04s
    552     0.00239     0.00216    1.10710  2.04s
    553     0.00236     0.00215    1.09408  2.04s
    554     0.00238     0.00215    1.11129  2.04s
    555     0.00237     0.00214    1.10382  2.04s
    556     0.00239     0.00214    1.11705  2.04s
    557     0.00237     0.00215    1.10337  2.04s
    558     0.00236     0.00215    1.09811  2.04s
    559     0.00236     0.00215    1.09803  2.04s
    560     0.00234     0.00214    1.09230  2.04s
    561     0.00235     0.00214    1.10069  2.04s
    562     0.00235     0.00215    1.09519  2.04s
    563     0.00237     0.00213    1.11101  2.04s
    564     0.00235     0.00213    1.10548  2.04s
    565     0.00235     0.00213    1.10277  2.04s
    566     0.00234     0.00212    1.10405  2.04s


...

   2260     0.00129     0.00118    1.08826  2.03s
   2261     0.00129     0.00118    1.09341  2.04s
   2262     0.00130     0.00119    1.09197  2.04s
   2263     0.00129     0.00119    1.08593  2.03s
   2264     0.00131     0.00118    1.11133  2.04s
   2265     0.00131     0.00118    1.11118  2.03s
   2266     0.00130     0.00118    1.10272  2.04s
   2267     0.00129     0.00118    1.09189  2.04s
   2268     0.00133     0.00119    1.11973  2.04s
   2269     0.00130     0.00118    1.09997  2.04s
   2270     0.00129     0.00118    1.09512  2.04s
   2271     0.00129     0.00118    1.09721  2.05s
   2272     0.00128     0.00118    1.08408  2.05s
   2273     0.00130     0.00118    1.09788  2.05s
   2274     0.00131     0.00118    1.10648  2.05s
   2275     0.00130     0.00118    1.10017  2.04s
   2276     0.00129     0.00118    1.09073  2.04s
   2277     0.00129     0.00118    1.08877  2.03s
   2278     0.00128     0.00118    1.08645  2.03s
   2279     0.00129     0.00118    1.09202  2.03s


   2419     0.00125     0.00116    1.07808  2.03s
   2420     0.00126     0.00116    1.08856  2.03s
   2421     0.00126     0.00115    1.08955  2.03s
   2422     0.00126     0.00115    1.09463  2.03s
   2423     0.00127     0.00115    1.10584  2.03s
   2424     0.00126     0.00115    1.08944  2.03s
   2425     0.00127     0.00116    1.09801  2.03s
   2426     0.00125     0.00115    1.08198  2.03s
   2427     0.00124     0.00115    1.07811  2.04s
   2428     0.00126     0.00115    1.09153  2.03s
   2429     0.00129     0.00115    1.11659  2.04s
   2430     0.00126     0.00115    1.09096  2.04s
   2431     0.00124     0.00115    1.07278  2.04s
   2432     0.00126     0.00116    1.08787  2.04s
   2433     0.00126     0.00116    1.08276  2.04s
   2434     0.00126     0.00116    1.08483  2.04s
   2435     0.00126     0.00115    1.08725  2.03s
   2436     0.00125     0.00115    1.08410  2.04s
   2437     0.00127     0.00115    1.10481  2.04s
   2438     0.00124     0.00115    1.07855  2.03s


   2577     0.00123     0.00113    1.08168  2.04s
   2578     0.00123     0.00113    1.08721  2.03s
   2579     0.00123     0.00113    1.09105  2.03s
   2580     0.00122     0.00113    1.08158  2.03s
   2581     0.00124     0.00113    1.10515  2.03s
   2582     0.00123     0.00113    1.08990  2.04s
   2583     0.00124     0.00113    1.09379  2.04s
   2584     0.00123     0.00113    1.08423  2.04s
   2585     0.00123     0.00113    1.08519  2.03s
   2586     0.00124     0.00113    1.09318  2.04s
   2587     0.00123     0.00113    1.08479  2.04s
   2588     0.00123     0.00113    1.08998  2.03s
   2589     0.00123     0.00113    1.08851  2.03s
   2590     0.00121     0.00113    1.07401  2.03s
   2591     0.00123     0.00113    1.09036  2.03s
   2592     0.00123     0.00113    1.08804  2.03s
   2593     0.00122     0.00113    1.08084  2.04s
   2594     0.00122     0.00113    1.08348  2.04s
   2595     0.00122     0.00112    1.08460  2.03s
   2596     0.00122     0.00112    1.08950  2.04s


   2735     0.00122     0.00111    1.09969  2.03s
   2736     0.00119     0.00111    1.07611  2.03s
   2737     0.00121     0.00111    1.09174  2.03s
   2738     0.00120     0.00111    1.08210  2.03s
   2739     0.00121     0.00111    1.09210  2.03s
   2740     0.00119     0.00111    1.07875  2.04s
   2741     0.00121     0.00111    1.09657  2.04s
   2742     0.00120     0.00111    1.08247  2.03s
   2743     0.00120     0.00110    1.08992  2.03s
   2744     0.00123     0.00110    1.10914  2.04s
   2745     0.00121     0.00111    1.09278  2.04s
   2746     0.00122     0.00111    1.09756  2.04s
   2747     0.00118     0.00111    1.06190  2.03s
   2748     0.00119     0.00111    1.07326  2.03s
   2749     0.00120     0.00111    1.08609  2.03s
   2750     0.00121     0.00111    1.09190  2.03s
   2751     0.00118     0.00110    1.07125  2.04s
   2752     0.00120     0.00110    1.08420  2.03s
   2753     0.00119     0.00111    1.07981  2.04s
   2754     0.00121     0.00111    1.09242  2.03s


   2893     0.00118     0.00109    1.07947  2.04s
   2894     0.00118     0.00109    1.08719  2.04s
   2895     0.00118     0.00109    1.07926  2.03s
   2896     0.00118     0.00109    1.08466  2.04s
   2897     0.00117     0.00109    1.07402  2.04s
   2898     0.00116     0.00109    1.06188  2.04s
   2899     0.00116     0.00109    1.06038  2.04s
   2900     0.00118     0.00109    1.07970  2.04s
   2901     0.00116     0.00109    1.06625  2.04s
   2902     0.00117     0.00109    1.07278  2.03s
   2903     0.00117     0.00109    1.07345  2.04s
   2904     0.00117     0.00109    1.07935  2.04s
   2905     0.00119     0.00109    1.09400  2.04s
   2906     0.00115     0.00109    1.06039  2.04s
   2907     0.00116     0.00109    1.06982  2.03s
   2908     0.00116     0.00109    1.07082  2.04s
   2909     0.00117     0.00109    1.07751  2.04s
   2910     0.00117     0.00109    1.07884  2.04s
   2911     0.00116     0.00109    1.06996  2.03s
   2912     0.00119     0.00109    1.09153  2.04s


In [16]:
import platform
platform.python_version()
#legg python 3.5.4

'3.5.4'